In [4]:
import numpy as np
array1 = np.array([1,2,3,4])
array2 = np.array([[1,2,3,4,5],
              [6,7,8,9,10],
              [11,12,13,14,15],
              [16,17,18,19,20]])
mask = [True, False, True, False]
print(array1[mask])
print(array2[mask])
print(array1[mask].shape)
print(array2[mask].shape)
print(array2[mask].shape)

[1 3]
[[ 1  2  3  4  5]
 [11 12 13 14 15]]
(2,)
(2, 5)
